s PROCEssING DATA TWEET MENGGUNAKAN KAMUs DATA KAMUs ALAY

In [25]:
#Menginmport Library Untuk s Processing

import pandas as pd # library untuk data manipulation
import re
import sqlite3

In [26]:
#Menginmport data tweet yang sudah di olah

df = pd.read_csv('tweet_data.csv')
df


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Total_HS,Label,Sentiment,total_kata
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,3,Toxic,Negative,25
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,0,Abusive,Negative,21
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,0,Netral,Positive,37
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,0,Netral,Positive,14
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,3,Toxic,Negative,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13039,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0,3,Toxic,Negative,10
13040,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0,0,Abusive,Negative,5
13041,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0,0,Netral,Positive,9
13042,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0,0,Netral,Positive,22


In [27]:
#Drop Kolom yang tidak di perlukan

df = df.drop(['HS_Individual','HS_Group','HS_Religion','HS_Race','HS_Physical','HS_Gender','HS_Other','HS_Weak','HS_Moderate','HS_Strong','total_kata','Total_HS'], axis=1)

In [28]:
df.head()

,Tweet,HS,Abusive,Label,Sentiment
0,- disaat semua cowok berusaha melacak perhatia...,1,1,Toxic,Negative
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,Abusive,Negative
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,Netral,Positive
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,Netral,Positive
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,Toxic,Negative


In [29]:
#Mengubah kata ke huruf kecil

def lowercase(s):
    return s.lower()


In [30]:
#Menghapus tanda baca

def punctuation(s):
    s = re.sub('[^0-9a-zA-Z]+', ' ', s) #menghilangkan semua karakter yang bukan huruf atau angka dan menggantinya dengan spasi.
    s = re.sub('^rt',' ', s) #menghapus awalan rt
    s = re.sub(r'\d+', '', s) #menghapus semua angka
    s = re.sub('user',' ', s) #menghapus kata 'user'
    s = re.sub(r':', ' ', s) #menggantikan karakter : dengan spasi
    s = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ', s) #menghapus semua URL 
    s = re.sub(' +', ' ', s) #menggantikan satu atau lebih spasi berturut-turut dengan satu spasi 
    s = re.sub('\n',' ',s) #menggantikan karakter newline (\n) dengan spasi 
    s = re.sub(r'pic.twitter.com.[\w]+', ' ', s) #menghapus semua tautan Twitter (pic.twitter.com)
    return s

In [31]:
#Membuat Koneksi ke database
conn = sqlite3.connect('database_alay.db')

In [32]:
#Import Kamusalay
kamusalay = pd.read_csv('new_kamusalay.csv',encoding='latin-1',names=['alay','normal'])

#Konek Kamus alay ke database

kamusalay.to_sql('kamusalay', conn, if_exists='replace',index=False)
kamusalay=pd.DataFrame(kamusalay)
kamusalay

,alay,normal
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit


In [33]:
#Mengambil Kembali kamusalay

db = sqlite3.connect('database_alay.db', check_same_thread = False)
q_kamusalay = 'SELECT * FROM kamusalay'
t_kamusalay = pd.read_sql_query(q_kamusalay, db)

In [34]:
alay_dict = dict(zip(t_kamusalay['alay'], t_kamusalay['normal']))
def alay_to_normal(s):
    for word in alay_dict:
        return ' '.join([alay_dict[word] if word in alay_dict else word for word in s.split(' ')])

In [35]:
# Copy Dataset

data = df.copy()
data.head()

,Tweet,HS,Abusive,Label,Sentiment
0,- disaat semua cowok berusaha melacak perhatia...,1,1,Toxic,Negative
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,Abusive,Negative
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,Netral,Positive
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,Netral,Positive
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,Toxic,Negative


In [36]:
#Membuat Fungsi Text Cleansing

def cleansing(s):
    s = lowercase(s)
    s = punctuation(s)
    s = alay_to_normal(s)
    return s

In [37]:
data['Tweet'] = data['Tweet'].apply(cleansing)
data.head()

,Tweet,HS,Abusive,Label,Sentiment
0,di saat semua cowok berusaha melacak perhatia...,1,1,Toxic,Negative
1,siapa yang telat memberi tau kamu edan sarap ...,0,1,Abusive,Negative
2,kadang aku berpikir kenapa aku tetap percaya ...,0,0,Netral,Positive
3,aku itu aku dan ku tau matamu sipit tapi dili...,0,0,Netral,Positive
4,kaum cebong kafir sudah kelihatan dongoknya d...,1,1,Toxic,Negative


In [38]:
#Membandingkan Tweet lama dan Tweet yang sudah di Cleansing


print('Tweet lama: {}'.format(df['Tweet'].iloc[4]))
print('\t')
print('Tweet baru: {}'.format(data['Tweet'].iloc[4]))

Tweet lama: USER USER Kaum cebong kapir udah keliatan dongoknya dari awal tambah dongok lagi hahahah'
	
Tweet baru:  kaum cebong kafir sudah kelihatan dongoknya dari awal tambah dungu lagi haha 


Menyiman data yang sudah dibersihkan

In [39]:
#Koneksi data yang sudah dibersihkan ke Database

data.to_sql('data', conn, if_exists='replace',index=False)
data=pd.DataFrame(data)
data

,Tweet,HS,Abusive,Label,Sentiment
0,di saat semua cowok berusaha melacak perhatia...,1,1,Toxic,Negative
1,siapa yang telat memberi tau kamu edan sarap ...,0,1,Abusive,Negative
2,kadang aku berpikir kenapa aku tetap percaya ...,0,0,Netral,Positive
3,aku itu aku dan ku tau matamu sipit tapi dili...,0,0,Netral,Positive
4,kaum cebong kafir sudah kelihatan dongoknya d...,1,1,Toxic,Negative
...,...,...,...,...,...
13039,jangan asal berbicara ndasmu congor kamu yang...,1,1,Toxic,Negative
13040,kasur mana enak kunyuk,0,1,Abusive,Negative
13041,hati hati bisu tidak dan lagi bosan duh xf xf...,0,0,Netral,Positive
13042,bom yang real mudah terdeteksi bom yang terku...,0,0,Netral,Positive


In [40]:
#Menutup Database
conn.commit()
conn.close()

In [41]:
data.head()

,Tweet,HS,Abusive,Label,Sentiment
0,di saat semua cowok berusaha melacak perhatia...,1,1,Toxic,Negative
1,siapa yang telat memberi tau kamu edan sarap ...,0,1,Abusive,Negative
2,kadang aku berpikir kenapa aku tetap percaya ...,0,0,Netral,Positive
3,aku itu aku dan ku tau matamu sipit tapi dili...,0,0,Netral,Positive
4,kaum cebong kafir sudah kelihatan dongoknya d...,1,1,Toxic,Negative


In [42]:
# #Menyimpan data ke csv untuk menampilkan teks dengan wordcloud
# data.to_csv('data_new.csv', index=False)